In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
import sqlite3
import os
from itertools import combinations
from datetime import datetime, timedelta

In [2]:
def generate_id(prefix, num):
    return f"{prefix}{num:03d}"

def random_string(length):
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    return ''.join(random.choice(letters) for i in range(length))

def query_db(query, params=()):
    conn = sqlite3.connect('clash_royale.db')
    df = pd.read_sql_query(query, conn, params=params)
    conn.close()
    return df

def generate_random_battle_duration():
    # Gerar uma duração aleatória em segundos entre 0 e 5 minutos (0 a 300 segundos)
    duration_seconds = random.randint(0, 300)
    
    # Converter para um objeto timedelta
    duration = timedelta(seconds=duration_seconds)
    
    return duration

In [3]:
descricoes_para_cartas = [
    "O Dragão Infernal queima tudo em seu caminho com um raio de fogo concentrado, deixando apenas cinzas.",
    "O Mago de Gelo congela seus inimigos com uma rajada de frio ártico, desacelerando seus movimentos.",
    "O Cavaleiro Fantasma emerge das sombras para causar dano letal antes de desaparecer sem deixar rastros.",
    "A Bruxa Sombria convoca morcegos das trevas enquanto lança feitiços devastadores contra seus oponentes.",
    "O Golem de Pedra avança lentamente, mas explode em mini-golems quando derrotado, continuando a batalha.",
    "A Princesa atira flechas flamejantes de longa distância, atingindo inimigos antes que eles a vejam.",
    "O Barril de Esqueletos lança um exército de esqueletos direto na linha de frente inimiga.",
    "O P.E.K.K.A. é uma máquina de combate blindada com uma espada gigante que corta qualquer coisa em seu caminho.",
    "O Gigante Real lança pedras enormes de sua besta, derrubando torres com facilidade.",
    "Os Três Mosqueteiros são uma força formidável, disparando uma chuva de balas que pode derrubar qualquer exército.",
    "O Lenhador enlouquecido derruba inimigos com seu machado, deixando uma poça de fúria ao ser derrotado.",
    "A Bandida desliza pelo campo de batalha, desferindo golpes críticos em suas vítimas antes de desaparecer.",
    "O Arqueiro Mágico dispara flechas que perfuram inimigos, atingindo vários alvos com um único tiro.",
    "O Mineiro cavouca subterraneamente para surgir onde menos se espera, causando dano diretamente às torres inimigas.",
    "O Mega Cavaleiro salta de inimigo em inimigo, esmagando-os com seu peso brutal.",
    "O Balão Esqueleto voa sobre as defesas, soltando bombas mortais diretamente nas torres adversárias.",
    "A Lava Hound libera uma enxurrada de lava-pups quando abatido, continuando o ataque aéreo.",
    "O Sparky carrega uma explosão elétrica devastadora, obliterando qualquer coisa que cruze seu caminho.",
    "Os Guardas Reais protegem suas torres com escudos robustos e lanças afiadas, enfrentando qualquer ameaça.",
    "O Mega Servo é uma criatura voadora blindada que ataca com garras afiadas, ideal para combates aéreos."
]

In [4]:
escala = 10
# Para cada grau desta escala são criados
# 4 players, 10 battles, 20 battleplayers, 20 decks, 160 deckcards
# Refiro-me, em cada um dos casos acima, a entidades distintas.
# O número de cartas distintas criadas é sempre 20

# Generate players
players = [
    {
        "playerTag": generate_id("PL", i),
        "name": f"Player{i}",
        "expLevel": random.randint(1, 13),
        "trophies": random.randint(0, 6000),
        "bestTrophies": random.randint(0, 6000),
        "wins": random.randint(0, 5000),
        "losses": random.randint(0, 5000),
        "battleCount": random.randint(0, 10000),
        "threeCrownWins": random.randint(0, 1000),
        "challengeCardsWon": random.randint(0, 1000),
        "challengeMaxWins": random.randint(0, 12),
        "tournamentCardsWon": random.randint(0, 1000),
        "tournamentBattleCount": random.randint(0, 1000),
        "role": random.choice(["member", "elder", "co-leader", "leader"]),
        "donations": random.randint(0, 5000),
        "donationsReceived": random.randint(0, 5000),
        "totalDonations": random.randint(0, 10000),
        "warDayWins": random.randint(0, 100),
        "clanCardsCollected": random.randint(0, 10000)
    }
    for i in range(1, escala * 4 + 1)
]

# Generate cards
cards = [
    {
        "cardId": generate_id("C", i),
        "name": f"Card{i}",
        "maxLevel": random.randint(1, 13),
        "iconUrl": f"http://example.com/card{i}.png",
        "rarity": random.choice(["common", "rare", "epic", "legendary"]),
        "description": descricoes_para_cartas[i-1]
    }
    for i in range(1, 21) # escala * 20 + 1
]

# Generate decks
decks = [
    {
        "deckId": generate_id("D", i)
    }
    for i in range(1, escala * 20 + 1)
]

# Generate deck cards ensuring each deck has 8 unique cards
deck_cards = []
for deck in decks:
#     card_ids = random.sample([card["cardId"] for card in cards], 8)
    card_ids = list(np.random.choice([card["cardId"] for card in cards], 8, replace=True))
    for card_id in card_ids:
        deck_cards.append({
            "deckCardId": generate_id("DC", len(deck_cards) + 1),
            "deckId": deck["deckId"],
            "cardId": card_id,
            "cardLevel": random.randint(1, 13)
        })

# Generate battles
# Não escolhe ainda o vencedor.
battles = [
    {
        "battleId": generate_id("B", i),
        "battleTime": datetime.now() - timedelta(days=random.randint(0, 365)),
        "gameMode": random.choice(["1v1", "2v2"]),
        "deckSelection": random.choice(["predefined", "custom"]),
#         "winner": random.choice([f"PL{j:03d}" for j in range(1, escala * 4 + 1)]),
        "battleDuration": generate_random_battle_duration(),
        "trophyChange": random.randint(-30, 30),
        "crowns": random.randint(0, 3),
        "arena": random.choice(["Training Camp", "Goblin Stadium", "Bone Pit", "Barbarian Bowl"])
    }
    for i in range(1, escala * 10 + 1)
]

# Generate battle players ensuring each battle has 2 players
battle_players = []
for battle in battles:
    participantes = random.sample(players, 2)
    for i in range(2):
        player = participantes[i]
        deck = random.choice(decks)
        battle_players.append({
            "battlePlayerId": generate_id("BP", len(battle_players) + 1),
            "battleId": battle["battleId"],
            "playerTag": player["playerTag"],
            "startingTrophies": random.randint(0, 6000),
            "crownsEarned": random.randint(0, 3),
            "kingTowerHitPoints": random.randint(0, 5000),
            "princessTowerHitPoints": f"{random.randint(0, 5000)},{random.randint(0, 5000)}",
            "clanName": f"Clan{random.randint(1, 10)}",
            "clanTag": random_string(5),
            "deckId": deck["deckId"]
        })
    battle['winner'] = random.choice([battle_players[-1]['playerTag'], battle_players[-2]['playerTag']])

# Apenas para facilitar visualização dos dados sintéticos criados:
df_players = pd.DataFrame(players)
df_cards = pd.DataFrame(cards)
df_decks = pd.DataFrame(decks)
df_deck_cards = pd.DataFrame(deck_cards)
df_battles = pd.DataFrame(battles)
df_battle_players = pd.DataFrame(battle_players)

In [5]:
# Esta célula cria o arquivo clash_royale.db com os dados sintéticos criados

# Primeiramente, apaga o arquivo clash_royale.db, caso existente, para criar um novo
if os.path.exists('clash_royale.db'):
    # Apaga o arquivo
    os.remove('clash_royale.db')

# Cria a conexão com o banco de dados
conn = sqlite3.connect('clash_royale.db')
cursor = conn.cursor()

# Cria as tabelas
cursor.execute('''
CREATE TABLE IF NOT EXISTS players (
    playerTag TEXT PRIMARY KEY,
    name TEXT,
    expLevel INTEGER,
    trophies INTEGER,
    bestTrophies INTEGER,
    wins INTEGER,
    losses INTEGER,
    battleCount INTEGER,
    threeCrownWins INTEGER,
    challengeCardsWon INTEGER,
    challengeMaxWins INTEGER,
    tournamentCardsWon INTEGER,
    tournamentBattleCount INTEGER,
    role TEXT,
    donations INTEGER,
    donationsReceived INTEGER,
    totalDonations INTEGER,
    warDayWins INTEGER,
    clanCardsCollected INTEGER
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS cards (
    cardId TEXT PRIMARY KEY,
    name TEXT,
    maxLevel INTEGER,
    iconUrl TEXT,
    rarity TEXT,
    description TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS decks (
    deckId TEXT PRIMARY KEY
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS deck_cards (
    deckCardId TEXT PRIMARY KEY,
    deckId TEXT,
    cardId TEXT,
    cardLevel INTEGER,
    FOREIGN KEY(deckId) REFERENCES decks(deckId),
    FOREIGN KEY(cardId) REFERENCES cards(cardId)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS battles (
    battleId TEXT PRIMARY KEY,
    battleTime TEXT,
    gameMode TEXT,
    deckSelection TEXT,
    battleDuration TEXT,
    winner TEXT,
    trophyChange INTEGER,
    crowns INTEGER,
    arena TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS battle_players (
    battlePlayerId TEXT PRIMARY KEY,
    battleId TEXT,
    playerTag TEXT,
    startingTrophies INTEGER,
    crownsEarned INTEGER,
    kingTowerHitPoints INTEGER,
    princessTowerHitPoints TEXT,
    clanName TEXT,
    clanTag TEXT,
    deckId TEXT,
    FOREIGN KEY(battleId) REFERENCES battles(battleId),
    FOREIGN KEY(playerTag) REFERENCES players(playerTag),
    FOREIGN KEY(deckId) REFERENCES decks(deckId)
)
''')

# Insere os dados nas tabelas
for player in players:
    cursor.execute('''
    INSERT INTO players VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(player.values()))

for card in cards:
    cursor.execute('''
    INSERT INTO cards VALUES (?, ?, ?, ?, ?, ?)
    ''', tuple(card.values()))

for deck in decks:
    cursor.execute('''
    INSERT INTO decks VALUES (?)
    ''', (deck['deckId'],))

for deck_card in deck_cards:
    cursor.execute('''
    INSERT INTO deck_cards VALUES (?, ?, ?, ?)
    ''', tuple(deck_card.values()))

for battle in battles:
    cursor.execute('''
    INSERT INTO battles VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        battle['battleId'],
        battle['battleTime'].isoformat(),
        battle['gameMode'],
        battle['deckSelection'],
        str(battle['battleDuration']),
        battle['winner'],
        battle['trophyChange'],
        battle['crowns'],
        battle['arena']
    ))

for battle_player in battle_players:
    cursor.execute('''
    INSERT INTO battle_players VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(battle_player.values()))

# Salva (commit) as mudanças e fecha a conexão com o banco de dados
conn.commit()
conn.close()


In [6]:
################################
####### TESTE DE QUERY 1 #######
################################

# Reabre a conexão com o banco de dados
conn = sqlite3.connect('clash_royale.db')
cursor = conn.cursor()

# Executa a consulta SQL
query = '''
SELECT *
FROM battle_players bp
INNER JOIN decks d ON bp.deckId = d.deckId;
'''
cursor.execute(query)

# Obtém os resultados da consulta
results = cursor.fetchall()

# Imprime os resultados
for row in results:
    print(row)

# Fecha a conexão com o banco de dados
conn.close()


('BP001', 'B001', 'PL037', 2503, 2, 1042, '2451,2389', 'Clan9', 'RMZWH', 'D049', 'D049')
('BP002', 'B001', 'PL001', 4679, 3, 769, '4890,3686', 'Clan4', 'KBNGO', 'D093', 'D093')
('BP003', 'B002', 'PL036', 1570, 3, 1616, '2772,520', 'Clan8', 'IYFRD', 'D039', 'D039')
('BP004', 'B002', 'PL024', 5159, 0, 4439, '5000,4527', 'Clan9', 'DLVRU', 'D039', 'D039')
('BP005', 'B003', 'PL019', 1735, 2, 2979, '3865,3834', 'Clan5', 'EKSCZ', 'D003', 'D003')
('BP006', 'B003', 'PL034', 3449, 1, 4520, '4583,2745', 'Clan10', 'BGJHD', 'D005', 'D005')
('BP007', 'B004', 'PL008', 3504, 2, 4570, '2579,914', 'Clan2', 'LBGEB', 'D047', 'D047')
('BP008', 'B004', 'PL009', 5192, 2, 3066, '160,2783', 'Clan6', 'XYRRJ', 'D119', 'D119')
('BP009', 'B005', 'PL005', 612, 2, 1409, '443,3719', 'Clan2', 'QVNLL', 'D014', 'D014')
('BP010', 'B005', 'PL033', 2824, 2, 289, '1109,4479', 'Clan9', 'GEKSE', 'D051', 'D051')
('BP011', 'B006', 'PL011', 1289, 0, 826, '173,2897', 'Clan10', 'PEKUK', 'D071', 'D071')
('BP012', 'B006', 'PL008', 3

In [7]:
################################
####### TESTE DE QUERY 2 #######
################################

# Função para obter deck_cards associados a um battle_player específico
def get_deck_cards_for_battle_player(battlePlayerId):
    # Reabre a conexão com o banco de dados
    conn = sqlite3.connect('clash_royale.db')
    cursor = conn.cursor()

    # Consulta SQL
    query = '''
    SELECT dc.*
    FROM battle_players bp
    INNER JOIN decks d ON bp.deckId = d.deckId
    INNER JOIN deck_cards dc ON d.deckId = dc.deckId
    WHERE bp.battlePlayerId = ?;
    '''

    # Executa a consulta com o battlePlayerId especificado
    cursor.execute(query, (battlePlayerId,))
    results = cursor.fetchall()

    # Fecha a conexão com o banco de dados
    conn.close()

    return results

# Exemplo de uso da função
battlePlayerId = 'BP001'  # Substitua pelo battlePlayerId desejado
deck_cards = get_deck_cards_for_battle_player(battlePlayerId)

# Imprime os resultados
for deck_card in deck_cards:
    print(deck_card)


('DC385', 'D049', 'C009', 8)
('DC386', 'D049', 'C006', 12)
('DC387', 'D049', 'C015', 4)
('DC388', 'D049', 'C005', 13)
('DC389', 'D049', 'C005', 7)
('DC390', 'D049', 'C013', 13)
('DC391', 'D049', 'C006', 12)
('DC392', 'D049', 'C004', 8)


In [8]:
################################
####### TESTE DE QUERY 3 #######
################################

# Reabre a conexão com o banco de dados
conn = sqlite3.connect('clash_royale.db')
cursor = conn.cursor()

# Executa a consulta SQL
query = '''
SELECT *
FROM battles bp
'''
cursor.execute(query)

# Obtém os resultados da consulta
results = cursor.fetchall()

# Fecha a conexão com o banco de dados
conn.close()

pd.DataFrame(results)

,0,1,2,3,4,5,6,7,8
0,B001,2024-02-01T00:58:09.729416,1v1,custom,0:03:26,PL037,15,3,Training Camp
1,B002,2024-04-07T00:58:09.729416,2v2,custom,0:00:11,PL024,22,1,Bone Pit
2,B003,2024-02-20T00:58:09.729416,1v1,custom,0:02:20,PL019,25,2,Bone Pit
3,B004,2023-11-22T00:58:09.729416,2v2,predefined,0:00:25,PL009,17,0,Training Camp
4,B005,2024-02-06T00:58:09.729416,1v1,custom,0:01:19,PL033,-14,0,Goblin Stadium
...,...,...,...,...,...,...,...,...,...
95,B096,2023-11-10T00:58:09.730412,1v1,custom,0:00:26,PL040,15,0,Bone Pit
96,B097,2023-11-07T00:58:09.730412,1v1,custom,0:03:21,PL024,26,3,Goblin Stadium
97,B098,2024-04-01T00:58:09.730412,2v2,custom,0:00:03,PL038,24,0,Goblin Stadium
98,B099,2023-11-02T00:58:09.730412,1v1,custom,0:04:40,PL013,25,2,Training Camp


<h3>Tarefa 1: Calcular a porcentagem de vitórias e derrotas utilizando a carta X em um intervalo de timestamps</h3>

In [10]:
def win_loss_percentage_with_card(card_id, start_time, end_time):
    query = '''
    SELECT bp.playerTag, b.winner
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE dc.cardId = ? AND b.battleTime BETWEEN ? AND ?
    '''
    df = query_db(query, (card_id, start_time, end_time))

    total_battles = len(df)
    if total_battles == 0:
        return {"win_percentage": 0, "loss_percentage": 0}

    wins = df[df['playerTag'] == df['winner']].shape[0]
    losses = total_battles - wins

    win_percentage = (wins / total_battles) * 100
    loss_percentage = (losses / total_battles) * 100

    return {"win_percentage": win_percentage, "loss_percentage": loss_percentage}

# Exemplo de uso:
card_id = 'C001'
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(win_loss_percentage_with_card(card_id, start_time, end_time))

{'win_percentage': 43.24324324324324, 'loss_percentage': 56.75675675675676}


<h3>Tarefa 2: Listar os decks completos que produziram mais de X% de vitórias em um intervalo de timestamps</h3>

In [11]:
def decks_with_high_win_rate(win_rate_threshold, start_time, end_time):
    query = '''
    SELECT bp.deckId, COUNT(*) as total_battles,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    WHERE b.battleTime BETWEEN ? AND ?
    GROUP BY bp.deckId
    HAVING (wins * 1.0 / total_battles) * 100 > ?
    '''
    df = query_db(query, (start_time, end_time, win_rate_threshold))
    return df

# Exemplo de uso:
win_rate_threshold = 50  # decks com mais de 50% de vitórias
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(decks_with_high_win_rate(win_rate_threshold, start_time, end_time))


   deckId  total_battles  wins
0    D008              1     1
1    D018              1     1
2    D021              3     2
3    D027              1     1
4    D029              1     1
5    D030              1     1
6    D031              1     1
7    D039              3     2
8    D041              2     2
9    D046              1     1
10   D048              1     1
11   D049              3     2
12   D051              1     1
13   D060              1     1
14   D064              1     1
15   D065              1     1
16   D069              1     1
17   D072              1     1
18   D074              1     1
19   D083              1     1
20   D084              1     1
21   D086              1     1
22   D087              1     1
23   D089              1     1
24   D094              1     1
25   D101              3     3
26   D104              1     1
27   D108              1     1
28   D109              3     2
29   D110              4     4
30   D113              1     1
31   D11

<h3>Tarefa 3: Calcular a quantidade de derrotas utilizando um combo de cartas em um intervalo de timestamps</h3>

In [12]:
def losses_with_card_combo(card_ids, start_time, end_time):
    card_placeholders = ', '.join('?' for _ in card_ids)
    query = f'''
    SELECT bp.battleId, COUNT(DISTINCT dc.cardId) as card_count
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE dc.cardId IN ({card_placeholders}) AND b.battleTime BETWEEN ? AND ?
    GROUP BY bp.battleId
    HAVING card_count = ?
    '''
    df = query_db(query, (*card_ids, start_time, end_time, len(card_ids)))

    total_battles = len(df)
    losses = total_battles - df[df['battleId'].isin(df['battleId'])].shape[0]

    return {"losses": losses}

# Exemplo de uso:
card_ids = ['C002', 'C003']
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(losses_with_card_combo(card_ids, start_time, end_time))


{'losses': 0}


<center><h3>Tarefa 3 versão 2 - utilizando apenas Python, sem SQL</h3></center>

In [13]:
card_ids = ['C002', 'C003']
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'

def derrotas_e_vitorias_do_combo(card_ids, start_time='1700-01-01T00:00:00', end_time='2200-01-01T00:00:00'):
    derrotas_do_combo = 0
    vitorias_do_combo = 0
    for batalha in df_battles[(df_battles['battleTime'] >= start_time) & (df_battles['battleTime'] <= end_time)]['battleId']:
        df_participantes = df_battle_players[df_battle_players['battleId']==batalha]
        vencedor = df_battles[df_battles['battleId']==batalha]['winner'].iloc[0]

        deque_perdedor = df_participantes.loc[df_participantes['playerTag'] != vencedor, 'deckId'].iloc[0]
        deque_vencedor = df_participantes.loc[df_participantes['playerTag'] == vencedor, 'deckId'].iloc[0]
        cartas_perdedoras = df_deck_cards[df_deck_cards['deckId']==deque_perdedor]['cardId'].tolist()
        cartas_vencedoras = df_deck_cards[df_deck_cards['deckId']==deque_vencedor]['cardId'].tolist()    
        if all([card in cartas_perdedoras for card in card_ids]):
            derrotas_do_combo+=1
        if all([card in cartas_vencedoras for card in card_ids]):
            vitorias_do_combo+=1
    return derrotas_do_combo, vitorias_do_combo

derrotas_do_combo, _ = derrotas_e_vitorias_do_combo(card_ids, start_time, end_time)
derrotas_do_combo

10

<h3>Tarefa 4: Calcular a quantidade de vitórias envolvendo a carta X onde o vencedor possui Z% menos troféus que o perdedor, a partida durou menos de 2 minutos, e o perdedor derrubou ao menos duas torres</h3>

In [14]:
def wins_with_card_under_conditions(card_id, trophy_diff_percentage, start_time, end_time):
    query = '''
    SELECT bp.battleId, bp.playerTag, bp.startingTrophies, b.winner, b.battleTime,
           (bp.kingTowerHitPoints + bp.princessTowerHitPoints) as totalHitPoints
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE dc.cardId = ? AND b.battleTime BETWEEN ? AND ? AND totalHitPoints < 2
    '''
    df = query_db(query, (card_id, start_time, end_time))
    
    # Filtrando as condições adicionais
    filtered_df = df[(df['winner'] == df['playerTag']) & 
                     ((df['startingTrophies'] * (1 - trophy_diff_percentage / 100)) > df['startingTrophies']) & 
                     (pd.to_datetime(df['battleTime']) - pd.to_datetime(df['battleTime']) < pd.Timedelta(minutes=2))]
    
    wins = filtered_df.shape[0]
    
    return {"wins": wins}

# Exemplo de uso:
card_id = 'C001'
trophy_diff_percentage = 20
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(wins_with_card_under_conditions(card_id, trophy_diff_percentage, start_time, end_time))


{'wins': 0}


<center><h3>Tarefa 4 versão 2 - utilizando apenas Python, sem SQL</h3></center>

In [15]:
# Exemplo de uso:
card_id = 'C001'
trophy_diff_percentage = 20
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'

#######
qtd_vitorias = 0
# Definir o limite de 2 minutos
two_minutes = timedelta(minutes=2)
# Criar o filtro
battle_duration_filter = df_battles['battleDuration'] <= two_minutes
for batalha in df_battles[(df_battles['battleTime'] >= start_time) & \
                          (df_battles['battleTime'] <= end_time) & \
                           battle_duration_filter]['battleId']:
    df_participantes = df_battle_players[df_battle_players['battleId']==batalha]
    vencedor = df_battles[df_battles['battleId']==batalha]['winner'].iloc[0]
    filtro_vencedor = df_participantes['playerTag']==vencedor
    trofeus_do_vencedor = df_participantes[filtro_vencedor]['startingTrophies'].iloc[0]
    trofeus_do_perdedor = df_participantes[~filtro_vencedor]['startingTrophies'].iloc[0]
    # Assumindo que o número de crownsEarned equivalha ao número de torres derrubadas:
    torres_derrubadas_pelo_perdedor = df_participantes[~filtro_vencedor]['crownsEarned'].iloc[0]
    vencedor_tem_menos_trofeus = trofeus_do_vencedor <= trofeus_do_perdedor * (1 - trophy_diff_percentage / 100)
    if vencedor_tem_menos_trofeus and (torres_derrubadas_pelo_perdedor >= 2):
        deque_vencedor = df_participantes.loc[df_participantes['playerTag'] == vencedor, 'deckId'].iloc[0]
        cartas_vencedoras = df_deck_cards[df_deck_cards['deckId']==deque_vencedor]['cardId'].tolist() 
        if card_id in cartas_vencedoras:
            qtd_vitorias+=1
qtd_vitorias

2

<h3>Tarefa 5: Listar o combo de cartas de tamanho N que produziram mais de Y% de vitórias em um intervalo de timestamps</h3>

In [16]:
def card_combos_with_high_win_rate(combo_size, win_rate_threshold, start_time, end_time):
    query = f'''
    SELECT bp.deckId, dc.cardId, COUNT(*) as total_battles,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE b.battleTime BETWEEN ? AND ?
    GROUP BY bp.deckId, dc.cardId
    HAVING COUNT(DISTINCT dc.cardId) = ? AND (wins * 1.0 / total_battles) * 100 > ?
    '''
    df = query_db(query, (start_time, end_time, combo_size, win_rate_threshold))
    return df

# Exemplo de uso:
combo_size = 2
win_rate_threshold = 50  # combos com mais de 50% de vitórias
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(card_combos_with_high_win_rate(combo_size, win_rate_threshold, start_time, end_time))


Empty DataFrame
Columns: [deckId, cardId, total_battles, wins]
Index: []


<center><h3>Tarefa 5 versão 2 - utilizando apenas Python, sem SQL</h3></center>

In [17]:
combo_size = 3
win_rate_threshold = 50  # combos com mais de 50% de vitórias
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'

# Esta célula exige paciência pois ela demora bastante para fornecer o resultado

def get_card_combinations(df_cards, N):
    card_ids = df_cards['cardId'].tolist()
    return list(combinations(card_ids, N))

# Exemplo de uso:
N = combo_size  # Substitua pelo número desejado de cartas na combinação
card_combinations = get_card_combinations(df_cards, N)

dicio_combos = {}

for combination in card_combinations:
    derrotas, vitorias = derrotas_e_vitorias_do_combo(combination,
                                                      start_time,
                                                      end_time)
    if (derrotas + vitorias) != 0:
        percent_vitoria = vitorias/(derrotas + vitorias)
    else:
        percent_vitoria = None
    dicio_combos[str(combination)] = percent_vitoria

df_combos = pd.DataFrame(dicio_combos, index=['pct_vitorias']).transpose()
df_combos[df_combos['pct_vitorias']>win_rate_threshold/100]

,pct_vitorias
"('C001', 'C002', 'C017')",1.0
"('C001', 'C003', 'C010')",0.8
"('C001', 'C003', 'C013')",0.545455
"('C001', 'C004', 'C008')",0.6
"('C001', 'C004', 'C010')",0.555556
...,...
"('C016', 'C017', 'C019')",0.6
"('C016', 'C017', 'C020')",0.714286
"('C017', 'C018', 'C019')",0.75
"('C017', 'C018', 'C020')",0.666667


<h3>Consultas adicionais I - Taxa de vitórias por raridade da carta</h3>
<p><i>Nesta consulta analisaremos o desempenho de cartas de diferentes raridades (comum, rara, épica, lendária) em todas as batalhas. A proposta é auxiliar a identificar se cartas de raridades específicas apresentam desempenho superior ou inferior.</i></p>

In [18]:
def win_rate_by_rarity():
    query = '''
    SELECT c.rarity, COUNT(*) as total_battles,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM cards c
    INNER JOIN deck_cards dc ON c.cardId = dc.cardId
    INNER JOIN battle_players bp ON dc.deckId = bp.deckId
    INNER JOIN battles b ON bp.battleId = b.battleId
    GROUP BY c.rarity
    '''
    df = query_db(query)
    
    df['win_rate'] = (df['wins'] / df['total_battles']) * 100
    return df

# Exemplo de uso:
print(win_rate_by_rarity())


      rarity  total_battles  wins   win_rate
0     common            194   114  58.762887
1       epic            151    77  50.993377
2  legendary            674   316  46.884273
3       rare            581   293  50.430293


<h3>Consultas adicionais II - Análise de diversidade de decks</h3>
<p><i>Nesta consulta examinaremos a variedade dos decks utilizados em partidas com alta taxa de troféus. A baixa diversidade de cartas no deck pode ser um indício de que certas combinações se tornam muito poderosas e precisam de equilíbrio.</i></p>

In [19]:
def deck_diversity(high_trophy_threshold):
    query = '''
    SELECT bp.deckId, COUNT(DISTINCT dc.cardId) as unique_cards
    FROM battle_players bp
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    INNER JOIN players p ON bp.playerTag = p.playerTag
    WHERE p.trophies > ?
    GROUP BY bp.deckId
    '''
    df = query_db(query, (high_trophy_threshold,))
    
    return df

# Exemplo de uso:
high_trophy_threshold = 5000  # Ajuste este valor conforme necessário
print(deck_diversity(high_trophy_threshold))


   deckId  unique_cards
0    D002             6
1    D003             7
2    D032             8
3    D039             7
4    D048             7
5    D049             6
6    D051             8
7    D060             8
8    D062             8
9    D072             4
10   D076             7
11   D078             6
12   D080             6
13   D101             6
14   D104             7
15   D109             6
16   D110             7
17   D112             8
18   D116             6
19   D119             6
20   D120             5
21   D124             6
22   D131             7
23   D132             8
24   D139             6
25   D158             7
26   D159             7
27   D161             8
28   D190             6


<h3>Consultas adicionais III - Taxa de uso do card vs taxa de vitórias</h3>
<p><i>Nesta consulta compararemos a frequência de uso de um card com sua taxa de ganho. Cards com altas taxas de utilização mas com baixas taxas de ganho podem ser consideradas mais fortes do que realmente são, enquanto cards com baixas taxas de utilização mas altas taxas de vitórias podem ser subvalorizados. </i></p>

In [20]:
def card_usage_vs_win_rate():
    query = '''
    SELECT c.cardId, c.name, c.rarity,
           COUNT(*) as usage_count,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM cards c
    INNER JOIN deck_cards dc ON c.cardId = dc.cardId
    INNER JOIN battle_players bp ON dc.deckId = bp.deckId
    INNER JOIN battles b ON bp.battleId = b.battleId
    GROUP BY c.cardId
    '''
    df = query_db(query)
    
    df['win_rate'] = (df['wins'] / df['usage_count']) * 100
    return df

# Exemplo de uso:
print(card_usage_vs_win_rate())


   cardId    name     rarity  usage_count  wins   win_rate
0    C001   Card1  legendary           74    32  43.243243
1    C002   Card2  legendary           68    25  36.764706
2    C003   Card3       epic           86    42  48.837209
3    C004   Card4  legendary          102    45  44.117647
4    C005   Card5  legendary           88    43  48.863636
5    C006   Card6       rare           75    42  56.000000
6    C007   Card7       rare           78    40  51.282051
7    C008   Card8  legendary          100    56  56.000000
8    C009   Card9     common           98    57  58.163265
9    C010  Card10     common           96    57  59.375000
10   C011  Card11  legendary           94    41  43.617021
11   C012  Card12       rare           63    27  42.857143
12   C013  Card13       rare           59    34  57.627119
13   C014  Card14  legendary           77    35  45.454545
14   C015  Card15       epic           65    35  53.846154
15   C016  Card16       rare           63    29  46.0317